In [2]:
# Imports
import pymongo
import pandas as pd
# from sklearn.model_selection import train_test_split
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.metrics import mean_squared_error
# from datetime import datetime
import joblib

In [3]:
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier

In [4]:
# import sys
# sys.path.append('d:\\GitHub\\NextTrip\\flask_LocationModel\\config')

In [5]:
import re

In [6]:
with open('/GitHub/NEXTTRIP/flask_LocationModel/config/db_config.py', 'r') as databaseUrlFile:
    DB_URL = databaseUrlFile.read()
DB_URL

match = re.search(r'"([^"]*)"', DB_URL)

if match:
    database_url = match.group(1)
else:
    print("No match found")

In [7]:
client = pymongo.MongoClient(database_url)

In [8]:
db = client['NextTripDB']
collection = db['rides_data']

In [9]:
# Retrieve the JSON data from MongoDB
data = list(collection.find({}))  
data_df = pd.DataFrame(data)
data_df.head()

,_id,Date/Time,Lat,Lon,__v
0,650db48407e33ccdd2398bfa,4/1/2014 0:17:00,40.7267,-74.0345,NaN
1,650db48407e33ccdd2398bfe,4/1/2014 0:33:00,40.7383,-74.0403,NaN
2,650db48407e33ccdd2398bff,4/1/2014 0:39:00,40.7223,-73.9887,NaN
3,650db48407e33ccdd2398c06,4/1/2014 2:11:00,40.6463,-73.7896,NaN
4,650db48407e33ccdd2398c09,4/1/2014 2:43:00,40.7580,-73.9761,NaN


In [10]:
# Load the dataset
# data = pd.read_csv('/GitHub/NEXTTRIP/flask_LocationModel/data/uber-raw-data-apr14.csv')

In [11]:
# Preprocess the data
data_df['Date/Time'] = pd.to_datetime(data_df['Date/Time'])

In [12]:
# Feature Engineering
data_df['weekday'] = data_df['Date/Time'].dt.weekday
data_df['hour'] = data_df['Date/Time'].dt.hour

In [13]:
dateTime_data = data_df[['weekday', 'hour']]
location_data = data_df[['Lat', 'Lon']]

In [14]:
num_clusters = 100

In [15]:
# Apply K-means clustering on the location data
kmeans = KMeans(n_clusters=num_clusters)
data_df['cluster'] = kmeans.fit_predict(location_data)

d:\GitHub\NextTrip\flask_LocationModel\env\lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [16]:
# train a model for each cluster
models= {}
for cluster_id in range(num_clusters):
    print('Training model for cluster: {}'.format(cluster_id))
    cluster_data = data_df[data_df['cluster'] == cluster_id]
    cluster_features  = cluster_data[['weekday', 'hour']]
    cluster_target = cluster_data['cluster']

    # train a random forest classifier
    model = RandomForestClassifier()
    model.fit(cluster_features, cluster_target)
    models[cluster_id] = model

Training model for cluster: 0
Training model for cluster: 1
Training model for cluster: 2
Training model for cluster: 3
Training model for cluster: 4
Training model for cluster: 5
Training model for cluster: 6
Training model for cluster: 7
Training model for cluster: 8
Training model for cluster: 9
Training model for cluster: 10
Training model for cluster: 11
Training model for cluster: 12
Training model for cluster: 13
Training model for cluster: 14
Training model for cluster: 15
Training model for cluster: 16
Training model for cluster: 17
Training model for cluster: 18
Training model for cluster: 19
Training model for cluster: 20
Training model for cluster: 21
Training model for cluster: 22
Training model for cluster: 23
Training model for cluster: 24
Training model for cluster: 25
Training model for cluster: 26
Training model for cluster: 27
Training model for cluster: 28
Training model for cluster: 29
Training model for cluster: 30
Training model for cluster: 31
Training model for

In [17]:
new_latitude = 40.7128  # Example latitude
new_longitude = -74.0060  # Example longitude
new_date = '1'  # Example date
new_time = '15'  # Example time

new_cluster = kmeans.predict([[new_latitude, new_longitude]])[0]
predicted_model = models[new_cluster]
predicted_cluster = predicted_model.predict([[new_date, new_time]])[0]

d:\GitHub\NextTrip\flask_LocationModel\env\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but KMeans was fitted with feature names
  warnings.warn(
d:\GitHub\NextTrip\flask_LocationModel\env\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [18]:
predicted_cluster

46

In [19]:
predicted_model = models[new_cluster]
predicted_cluster = predicted_model.predict([[new_date, new_time]])[0]
    
# Retrieve the centroid (mean) of the cluster for latitude and longitude
cluster_centroid = kmeans.cluster_centers_[predicted_cluster]
predicted_latitude, predicted_longitude = cluster_centroid[0], cluster_centroid[1]

d:\GitHub\NextTrip\flask_LocationModel\env\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [20]:
predicted_latitude

40.71062847352025

In [21]:
predicted_longitude

-74.00519775700934

In [25]:
# Persist the models
for cluster_id, model in models.items():
    joblib.dump(model, f'/GitHub/NEXTTRIP/flask_LocationModel/models/clusters/cluster_{cluster_id}_model.pkl')

In [ ]:
#Save model columns
# rnd_columns = list(X_train.columns)
# joblib.dump(rnd_columns, '/GitHub/NEXTTRIP/flask_LocationModel/models/rnd_columns.pkl')